<div style="font-size:18pt; padding-top:20px; text-align:center">СЕМИНАР. <b>Статистика и <span style="font-weight:bold; color:green">NumPy/SciPy</span>. Задачи</b></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@yandex.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ul>
        <li><a href="#1">Генеральные совокупности</a></li>
        <li><a href="#2">Задача 1. Доверительный интервал</a></li>
        <li><a href="#3">Задача 2. Проверка гипотезы с одной выборкой</a>
        <li><a href="#4">Задача 3. Проверка гипотезы с парной выборкой</a>
        <li><a href="#5">Задача 4. Проверка гипотезы по двум выборкам</a>
        </li>
    </ul>
</div>

<p><b>Подключение библиотек и функций</b></p>

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def getSE(sigma, n):
    return sigma / np.sqrt(n)

def getSE2(sigma1, n1, sigma2, n2):
    return np.sqrt(sigma1**2 / n1 + sigma2**2 / n2)

def getZ(x, mu, se):
    return (x - mu) / se

def getPrByX(x, mu, se):
    return stats.norm.cdf(x, loc=mu, scale=se)

def getPrByZ(z):
    return stats.norm.cdf(z, loc=0, scale=1)

def getZbyPr2Tail(pr):
    return stats.norm.ppf((1-pr)/2, loc=0, scale=1)

def getZbyPr1Tail(pr):
    return stats.norm.ppf(1-pr, loc=0, scale=1)

def getPValue1Tail_Norm(z):
    return 1 - stats.norm.cdf(z, loc=0, scale=1)

def getPValue2Tail_Norm(z):
    return 2 * (stats.norm.cdf(-abs(z), loc=0, scale=1))

def getPValue2Tail_T(t, df):
    return 2 * stats.t.cdf(-abs(t), df=df)

def getPValue1Tail_T(t, df):
    return 1 - stats.t.cdf(t, df=df)

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">Генеральные совокупности</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>Создание генеральных совокупностей</p>

In [ ]:
mu1 = 70
sigma1 = 30
population1 = stats.norm.rvs(size=10000, loc=mu1, scale=sigma1)

In [ ]:
mu2 = 60
sigma2 = 20
population2 = stats.norm.rvs(size=10000, loc=mu2, scale=sigma2)

In [ ]:
mu3 = 2
sigma3 = 2
diff = stats.norm.rvs(size=10000, loc=mu3, scale=sigma3)
population3 = list(zip(population1, population1 + diff))

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">Задача 1. Доверительный интервал</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [ ]:
n = 40
sample_2015 = np.random.choice(population1, n, replace=False)
sample_2015

<p>Статистики выборки</p>

In [ ]:
minV, maxV, x_bar_2015, var_2015, sk, kur = stats.describe(sample_2015)
print(x_bar_2015)
print(var_2015)

<p>или</p>

In [ ]:
#Среднее значение
x_bar = sample_2015.mean()
print(x_bar)

#Выборочная дисперсия
var_2015 = sample_2015.var(ddof=1)
print(var_2015)

#Стандартное отклонение (несмещенное)
s_2015 = sample_2015.std(ddof=1)
print(s_2015)

<p><b><i>Доверительный интервал</i></b></p>

In [ ]:
#Уровни доверия: 95%, 90%, 80%
prs = np.array([0.95, 0.9, 0.8])

In [ ]:
#Стандартная ошибка
se = getSE(s_2015, n)
se

In [ ]:
#Доверительный интервал
lowers, uppers = stats.norm.interval(prs, loc=x_bar, scale=se)
for i in range(len(prs)):
    print("Уровень доверия %i%%: (%f, %f)" % (prs[i]*100, lowers[i], uppers[i]))

In [ ]:
#Доверительный интервал для t-распределения (при небольших n)
lowers_t, uppers_t = stats.t.interval(prs, df=n-1, loc=x_bar, scale=se)
print("t-распределение")
for i in range(len(prs)):
    print("Уровень доверия %i%%: (%f, %f)" % (prs[i]*100, lowers_t[i], uppers_t[i]))

<p>или</p>

In [ ]:
#z-значения для двустороннего интервала
zs = stats.norm.ppf((1-prs)/2, loc=0, scale=1)
zs

In [ ]:
#z-значения для двустороннего интервала
zs = getZbyPr2Tail(prs)
zs

In [ ]:
#Предельная ошибка
delta = abs(zs*se)
delta

In [ ]:
#Доверительный интервал
lowers = x_bar - delta
uppers = x_bar + delta

for i in range(len(prs)):
    print("Уровень доверия %i%%: (%f, %f)" % (prs[i]*100, lowers[i], uppers[i]))

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">Задача 2. Проверка гипотезы с одной выборкой</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>$$H_0: \mu_{2014}=\mu_{2015}$$<p>
<p>$$H_A: \mu_{2014} \neq \mu_{2015}$$<p>

<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [ ]:
mu_2014 = 65
n = 40
alpha = 0.05 #уровень значимости
sample_2015 = np.random.choice(population1, n, replace=False)
sample_2015

In [ ]:
#Среднее значение
x_bar_2015 = sample_2015.mean()
print(x_bar)

#Стандартное отклонение (несмещенное)
s_2015 = sample_2015.std(ddof=1)
print(s_2015)

#Стандартная ошибка
se = getSE(s_2015, n)
print(se)

<p><b>Проверка гипотезы</b></p>

In [ ]:
#Z-score
z = getZ(x_bar_2015, mu_2014, se)
z

In [ ]:
#P-Value 
pvalue = getPValue2Tail_Norm(z)
pvalue

In [ ]:
#P-Value для t-распределения
df = n - 1 #степень свободы
pvalue_t = getPValue2Tail_T(z, df)
pvalue_t

<p>или</p>

In [ ]:
tvalue, pvalue_t = stats.ttest_1samp(sample_2015, mu_2014)
pvalue_t

In [ ]:
pvalue_t = 2 * stats.t.cdf(-abs(tvalue), df)
pvalue_t

<p><b>Проверка гипотезы на соответствие уровню значимости $\alpha$</b></p>

In [ ]:
if alpha > pvalue:
    print("alpha > p-value")
    print("Отказываемся от нулевую гипотезы в пользу альтернативной")
else:
    print("alpha <= p-value")
    print("Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)")

<p><b>График</b></p>

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">Задача 3. Проверка гипотезы с парной выборкой</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [ ]:
alpha = 0.05 #уровень значимости

In [ ]:
import random

n_items = 40
sample_items = np.array(random.sample(population3, n_items))
sample_items

In [ ]:
sample_diff = sample_items[:,0] - sample_items[:,1]
sample_diff

In [ ]:
#Среднее значение
x_bar_diff = sample_diff.mean()
print("x_bar_diff =", x_bar_diff)

#Стандартное отклонение (несмещенное)
s_diff = sample_diff.std(ddof=1)
print("s_diff =", s_diff)

#Стандартная ошибка
se_diff = getSE(s_diff, n_items)
print("se_diff =", se_diff)

<p><b>Проверка гипотезы</b></p>

<p>$$H_0: \mu_{diff}=0$$<p>
<p>$$H_A: \mu_{diff} \neq 0$$<p>

In [ ]:
#Z-score
z = getZ(x_bar_diff, 0, se_diff)
z

In [ ]:
#P-Value 
pvalue = getPValue2Tail_Norm(z)
pvalue

In [ ]:
#P-Value для t-распределения
df = n_items - 1
   
pvalue_t = getPValue2Tail_T(z, df)
pvalue_t

<p>или</p>

In [ ]:
tvalue, pvalue_t = stats.ttest_rel(sample_items[:,0], sample_items[:,1])
print("t-value =", tvalue)
print("p-value =", pvalue_t)

In [ ]:
pvalue_t = 2 * stats.t.cdf(-abs(tvalue), df)
pvalue_t

<p><b>Проверка гипотезы на соответствие уровню значимости $\alpha$</b></p>

In [ ]:
if alpha > pvalue:
    print("alpha > p-value")
    print("Отказываемся от нулевую гипотезы в пользу альтернативной")
else:
    print("alpha <= p-value")
    print("Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)")

<p><b>График</b></p>

<a name="5"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">Задача 4. Проверка гипотезы по двум выборкам</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [ ]:
alpha = 0.05 #уровень значимости

In [ ]:
n_london = 40
sample_london = np.random.choice(population1, n_london, replace=False)
sample_london

In [ ]:
n_moscow = 50
sample_moscow = np.random.choice(population2, n_moscow, replace=False)
sample_moscow

In [ ]:
#Среднее значение
x_bar_london = sample_london.mean()
x_bar_moscow = sample_moscow.mean()
print("x_bar_london =", x_bar_london)
print("x_bar_moscow =", x_bar_moscow)

#Стандартное отклонение (несмещенное)
s_london = sample_london.std(ddof=1)
s_moscow = sample_moscow.std(ddof=1)
print("s_london =", s_london)
print("s_moscow =", s_moscow)

#Стандартная ошибка
se_london_moscow = getSE2(s_london, n_london, s_moscow, n_moscow)
print("se_london_moscow =", se_london_moscow)

<p><b>Проверка гипотезы</b></p>

<p>$$H_0: \mu_{treated}-\mu_{control}=0$$<p>
<p>$$H_A: \mu_{treated}-\mu_{control} \neq 0$$<p>

In [ ]:
#Z-score
z = getZ(x_bar_london-x_bar_moscow, 0, se_london_moscow)
z

In [ ]:
#P-Value 
pvalue = getPValue2Tail_Norm(z)
pvalue

In [ ]:
#P-Value для t-распределения
df = n_london - 1
if n_london > n_moscow:
    df = n_moscow - 1
    
pvalue_t = getPValue2Tail_T(z, df)
pvalue_t

<p>или</p>

In [ ]:
tvalue, pvalue_t = stats.ttest_ind(sample_london, sample_moscow, equal_var=False)
pvalue_t

<p>или</p>

In [ ]:
tvalue, pvalue_t = stats.ttest_ind_from_stats(x_bar_london, s_london, n_london, x_bar_moscow, s_moscow, n_moscow, equal_var=False)
pvalue_t

In [ ]:
pvalue_t = 2 * stats.t.cdf(-abs(tvalue), df)
pvalue_t

<p><b>Проверка гипотезы на соответствие уровню значимости $\alpha$</b></p>

In [ ]:
if alpha > pvalue:
    print("alpha > p-value")
    print("Отказываемся от нулевую гипотезы в пользу альтернативной")
else:
    print("alpha <= p-value")
    print("Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)")

<p><b>График</b></p>